# Pronóstico de la evolución de pacientes con diabetes

## Descripción del problema real

Los tratamientos médicos están basados en las expectativas de recuperación o el avance de una enfermedad para tomar decisiones. En este caso, un equipo médico desea contar con pronósticos de pacientes con diabetes para tomar decisiones sobre su tratamiento.

## Descripción del problema en términos de los datos

Se desea determinar el progreso de la diabeteis un año hacia adelante a partir de las variables medidas para 442 pacientes. La información está almacenada en el archivo `datos/diabetes.csv`. Las variables medidas son: edad, sexo, indice de masa corporal, presión sanguinea y seis medidas de serum en la sangre. Se desea pronósticar el progreso de la enfermedad a partir de las variables dadas.

## Aproximaciones posibles

En este caso, se desea comparar los resultados de un modelo de regresión lineal y un modelo de redes neuronales artificiales.

## Requerimientos

Usted debe:

* Determinar cuáles de las variables consideradas son relevantes para el problema.


* Determinar si hay alguna transformación de las variables de entrada o de salida que mejore el pronóstico del modelo.


* Construir un modelo de regresión lineal que sirva como base para construir un modelo de redes neuronales artificiales.



* Construir un modelo de redes neuronales artificiales. Asimismo, debe determinar el número de neuronas en la capa o capas ocultas.


* Utiizar una técnica como crossvalidation u otra similar para establecer la robustez del modelo.

## Lectura y limpieza de datos

In [70]:
#Librerias
import sys
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import metrics
import numpy as np
from collections import OrderedDict
import itertools
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

In [5]:
#Lectura de datos
datos = pd.read_csv("./datos/diabetes.csv")
datos

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,Y
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930,220.0


In [6]:
#Separamos variables dependientes de independientes
##Selección de variables
X = datos.drop(["Y"],axis = 1)
#Y = datos["Y"]
frame = { 'y': datos["Y"] } 
Y = pd.DataFrame(frame)
Y

,y
0,151.0
1,75.0
2,141.0
3,206.0
4,135.0
...,...
437,178.0
438,104.0
439,132.0
440,220.0


### Normalizamos todos los datos a valores entre cero y uno

In [7]:
#Normalizamos todos los datos
scalerX = MinMaxScaler()
scaled_dfX = scalerX.fit_transform(X)
datos_normX = pd.DataFrame(scaled_dfX, columns=['age', 'sex', 'bmi','bp','s1','s2','s3','s4','s5','s6'])
scalerY = MinMaxScaler()
scaled_dfY = scalerY.fit_transform(Y)
datos_normY = pd.DataFrame(scaled_dfY,columns=['y'])




##### Utilizamos el criterio de Akaike para comparar distintos modelos generados y ver que variables son relevantes.
##### Para esto, utilizamos la estrategia forward para recorrer las variables,lo hacemos construyendo modelos con todas las combinaciones de variables posibles y escogeremos el que menor AIC tenga.

In [21]:
#Generamos la combinación de todas las variables
df_lagged=data.copy()
lag_combinations = OrderedDict()
l = list(range(1,11,1))
for i in range(1, 11, 1):
   for combination in itertools.combinations(l, i):
      lag_combinations[combination] = 0.0

min_aic = sys.float_info.max

best_olsr_model_results = None

list_columns = ['age',	'sex',	'bmi',	'bp',	's1',	's2',	's3',	's4',	's5',	's6']


for combination in lag_combinations:
  actual_columns = []
  for num in combination:
    actual_columns.append(list_columns[num-1])

    
  print("Variables actuales:")
  print(actual_columns)

    
    #Utilizamos OLS de statmodels
  olsr_results = sm.OLS(datos_normY, datos_normX[actual_columns].copy()).fit()

  lag_combinations[combination] = olsr_results.aic
  print('AIC='+str(lag_combinations[combination]))
  if olsr_results.aic < min_aic:
    min_aic = olsr_results.aic
    best_olsr_model_results = olsr_results

Variables actuales:
['age']
AIC=79.51027446787214
Variables actuales:
['sex']
AIC=377.1218844769495
Variables actuales:
['bmi']
AIC=-151.46559865766005
Variables actuales:
['bp']
AIC=-71.67860456169922
Variables actuales:
['s1']
AIC=36.71650954109248
Variables actuales:
['s2']
AIC=70.50951869936296
Variables actuales:
['s3']
AIC=290.7107395119515
Variables actuales:
['s4']
AIC=25.092675579800584
Variables actuales:
['s5']
AIC=-174.39333507281947
Variables actuales:
['s6']
AIC=-59.557527655055196
Variables actuales:
['age', 'sex']
AIC=76.25943439494438
Variables actuales:
['age', 'bmi']
AIC=-182.62405757862223
Variables actuales:
['age', 'bp']
AIC=-82.79795965700384
Variables actuales:
['age', 's1']
AIC=-1.1004160630374145
Variables actuales:
['age', 's2']
AIC=12.682522759879134
Variables actuales:
['age', 's3']
AIC=80.01866811340699
Variables actuales:
['age', 's4']
AIC=-59.42750022391135
Variables actuales:
['age', 's5']
AIC=-174.538539076966
Variables actuales:
['age', 's6']
AIC=-68.

AIC=-144.60812286269663
Variables actuales:
['age', 'bp', 's4', 's5']
AIC=-214.9566907057282
Variables actuales:
['age', 'bp', 's4', 's6']
AIC=-167.91168083909736
Variables actuales:
['age', 'bp', 's5', 's6']
AIC=-210.10789170830947
Variables actuales:
['age', 's1', 's2', 's3']
AIC=-17.823167739255155
Variables actuales:
['age', 's1', 's2', 's4']
AIC=-87.87923108068105
Variables actuales:
['age', 's1', 's2', 's5']
AIC=-198.3290397109173
Variables actuales:
['age', 's1', 's2', 's6']
AIC=-72.87512001035668
Variables actuales:
['age', 's1', 's3', 's4']
AIC=-72.75352816298926
Variables actuales:
['age', 's1', 's3', 's5']
AIC=-178.84537074179877
Variables actuales:
['age', 's1', 's3', 's6']
AIC=-94.76340398331183
Variables actuales:
['age', 's1', 's4', 's5']
AIC=-186.06100884934006
Variables actuales:
['age', 's1', 's4', 's6']
AIC=-113.77590082900804
Variables actuales:
['age', 's1', 's5', 's6']
AIC=-190.58242916138659
Variables actuales:
['age', 's2', 's3', 's4']
AIC=-85.36746048851512
Var

AIC=-208.69996567596957
Variables actuales:
['age', 'bmi', 's1', 's3', 's4']
AIC=-212.25872432840526
Variables actuales:
['age', 'bmi', 's1', 's3', 's5']
AIC=-281.2464643458385
Variables actuales:
['age', 'bmi', 's1', 's3', 's6']
AIC=-215.8476059957976
Variables actuales:
['age', 'bmi', 's1', 's4', 's5']
AIC=-282.6205283343396
Variables actuales:
['age', 'bmi', 's1', 's4', 's6']
AIC=-224.0699747217244
Variables actuales:
['age', 'bmi', 's1', 's5', 's6']
AIC=-281.01925074845497
Variables actuales:
['age', 'bmi', 's2', 's3', 's4']
AIC=-226.62807167322137
Variables actuales:
['age', 'bmi', 's2', 's3', 's5']
AIC=-278.7333840764792
Variables actuales:
['age', 'bmi', 's2', 's3', 's6']
AIC=-211.61399988785922
Variables actuales:
['age', 'bmi', 's2', 's4', 's5']
AIC=-278.5924599313249
Variables actuales:
['age', 'bmi', 's2', 's4', 's6']
AIC=-233.79329293039302
Variables actuales:
['age', 'bmi', 's2', 's5', 's6']
AIC=-274.15472318343893
Variables actuales:
['age', 'bmi', 's3', 's4', 's5']
AIC=-

['sex', 's1', 's3', 's4', 's6']
AIC=-115.12508359653236
Variables actuales:
['sex', 's1', 's3', 's5', 's6']
AIC=-199.88395691455105
Variables actuales:
['sex', 's1', 's4', 's5', 's6']
AIC=-205.50270379419146
Variables actuales:
['sex', 's2', 's3', 's4', 's5']
AIC=-176.83199647983645
Variables actuales:
['sex', 's2', 's3', 's4', 's6']
AIC=-124.69351594730381
Variables actuales:
['sex', 's2', 's3', 's5', 's6']
AIC=-198.31231888882087
Variables actuales:
['sex', 's2', 's4', 's5', 's6']
AIC=-196.86361982628785
Variables actuales:
['sex', 's3', 's4', 's5', 's6']
AIC=-198.90165416020147
Variables actuales:
['bmi', 'bp', 's1', 's2', 's3']
AIC=-246.36284263554182
Variables actuales:
['bmi', 'bp', 's1', 's2', 's4']
AIC=-269.6424692499901
Variables actuales:
['bmi', 'bp', 's1', 's2', 's5']
AIC=-299.74813778015175
Variables actuales:
['bmi', 'bp', 's1', 's2', 's6']
AIC=-231.4909211531243
Variables actuales:
['bmi', 'bp', 's1', 's3', 's4']
AIC=-247.61954790403377
Variables actuales:
['bmi', 'bp', 

AIC=-190.0260012346306
Variables actuales:
['age', 'sex', 'bp', 's1', 's5', 's6']
AIC=-226.48020651626587
Variables actuales:
['age', 'sex', 'bp', 's2', 's3', 's4']
AIC=-181.32946141767923
Variables actuales:
['age', 'sex', 'bp', 's2', 's3', 's5']
AIC=-234.30946128448716
Variables actuales:
['age', 'sex', 'bp', 's2', 's3', 's6']
AIC=-159.8042186521758
Variables actuales:
['age', 'sex', 'bp', 's2', 's4', 's5']
AIC=-234.81387971368179
Variables actuales:
['age', 'sex', 'bp', 's2', 's4', 's6']
AIC=-199.01141465688022
Variables actuales:
['age', 'sex', 'bp', 's2', 's5', 's6']
AIC=-214.6869382039646
Variables actuales:
['age', 'sex', 'bp', 's3', 's4', 's5']
AIC=-236.4704737737785
Variables actuales:
['age', 'sex', 'bp', 's3', 's4', 's6']
AIC=-187.89752846491626
Variables actuales:
['age', 'sex', 'bp', 's3', 's5', 's6']
AIC=-243.8355113284225
Variables actuales:
['age', 'sex', 'bp', 's4', 's5', 's6']
AIC=-225.00493613788683
Variables actuales:
['age', 'sex', 's1', 's2', 's3', 's4']
AIC=-93.5

AIC=-291.8626246599034
Variables actuales:
['sex', 'bmi', 's1', 's3', 's4', 's5']
AIC=-285.33360109937155
Variables actuales:
['sex', 'bmi', 's1', 's3', 's4', 's6']
AIC=-226.87245469833726
Variables actuales:
['sex', 'bmi', 's1', 's3', 's5', 's6']
AIC=-288.1899206338561
Variables actuales:
['sex', 'bmi', 's1', 's4', 's5', 's6']
AIC=-290.1063329690779
Variables actuales:
['sex', 'bmi', 's2', 's3', 's4', 's5']
AIC=-280.31504777479597
Variables actuales:
['sex', 'bmi', 's2', 's3', 's4', 's6']
AIC=-239.28177750417547
Variables actuales:
['sex', 'bmi', 's2', 's3', 's5', 's6']
AIC=-285.37078312294955
Variables actuales:
['sex', 'bmi', 's2', 's4', 's5', 's6']
AIC=-284.52234986370706
Variables actuales:
['sex', 'bmi', 's3', 's4', 's5', 's6']
AIC=-283.66201072961996
Variables actuales:
['sex', 'bp', 's1', 's2', 's3', 's4']
AIC=-182.46757371776005
Variables actuales:
['sex', 'bp', 's1', 's2', 's3', 's5']
AIC=-253.90772919116569
Variables actuales:
['sex', 'bp', 's1', 's2', 's3', 's6']
AIC=-171.1

AIC=-243.2905676423237
Variables actuales:
['age', 'sex', 'bp', 's1', 's3', 's4', 's6']
AIC=-188.13764183794967
Variables actuales:
['age', 'sex', 'bp', 's1', 's3', 's5', 's6']
AIC=-243.90919427173924
Variables actuales:
['age', 'sex', 'bp', 's1', 's4', 's5', 's6']
AIC=-248.79575203442357
Variables actuales:
['age', 'sex', 'bp', 's2', 's3', 's4', 's5']
AIC=-235.97630643891955
Variables actuales:
['age', 'sex', 'bp', 's2', 's3', 's4', 's6']
AIC=-197.17810235956165
Variables actuales:
['age', 'sex', 'bp', 's2', 's3', 's5', 's6']
AIC=-241.87282000143125
Variables actuales:
['age', 'sex', 'bp', 's2', 's4', 's5', 's6']
AIC=-237.96750295593017
Variables actuales:
['age', 'sex', 'bp', 's3', 's4', 's5', 's6']
AIC=-242.3613639661056
Variables actuales:
['age', 'sex', 's1', 's2', 's3', 's4', 's5']
AIC=-203.48492205710397
Variables actuales:
['age', 'sex', 's1', 's2', 's3', 's4', 's6']
AIC=-135.11965162539116
Variables actuales:
['age', 'sex', 's1', 's2', 's3', 's5', 's6']
AIC=-214.8043644402179


AIC=-254.65468404551132
Variables actuales:
['age', 'sex', 'bp', 's1', 's2', 's4', 's5', 's6']
AIC=-254.42101623698716
Variables actuales:
['age', 'sex', 'bp', 's1', 's3', 's4', 's5', 's6']
AIC=-247.82443317395666
Variables actuales:
['age', 'sex', 'bp', 's2', 's3', 's4', 's5', 's6']
AIC=-241.8492804875001
Variables actuales:
['age', 'sex', 's1', 's2', 's3', 's4', 's5', 's6']
AIC=-212.86959650624397
Variables actuales:
['age', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5']
AIC=-296.85623735220304
Variables actuales:
['age', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's6']
AIC=-267.96565686607516
Variables actuales:
['age', 'bmi', 'bp', 's1', 's2', 's3', 's5', 's6']
AIC=-297.20897623598523
Variables actuales:
['age', 'bmi', 'bp', 's1', 's2', 's4', 's5', 's6']
AIC=-296.0950195989359
Variables actuales:
['age', 'bmi', 'bp', 's1', 's3', 's4', 's5', 's6']
AIC=-296.9407314640158
Variables actuales:
['age', 'bmi', 'bp', 's2', 's3', 's4', 's5', 's6']
AIC=-295.1206862502612
Variables actuales:
['age', 'b

### Resultados del mejor modelo
#### Según los resultados las variables más relevantes para el problema son: (sex,bmi,bp,s1,s4,s5). Criterio Menor AIC.

In [22]:
#Mejor modelo resultante
print(best_olsr_model_results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.869
Model:                            OLS   Adj. R-squared (uncentered):              0.868
Method:                 Least Squares   F-statistic:                              483.6
Date:                Tue, 07 Apr 2020   Prob (F-statistic):                   3.59e-189
Time:                        23:42:57   Log-Likelihood:                          162.97
No. Observations:                 442   AIC:                                     -313.9
Df Residuals:                     436   BIC:                                     -289.4
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

##### Dividimos los datos para obtener el conjunto de entrenamiento y de pronóstico (80% entrenamiento,20% pronóstico)

In [23]:
#division de los datos
X_train, X_test, y_train, y_test = train_test_split(datos_normX,datos_normY,test_size=0.20)

In [10]:
#transformaciones de los datos
transformaciones = [
  ("Logaritmo", lambda x: np.log(x)),
  ("Exponencial", lambda x: np.exp(x)),
  ("Identidad", lambda x: x),
  ("Raiz cuadrada", lambda x: x**(1/2)),
  ("Raiz cúbica", lambda x: x**(1/3))
]
#Para x
tf_transformX=[]
for transf,func in transformaciones:
    data = datos_normX.copy()
    data = data.apply(func)
    tf_transformX.append((transf,data))
print(tf_transformX)

#Para y
tf_transformY=[]
for transf,func in transformaciones:
    data = datos_normY.copy()
    data = data.apply(func)
    tf_transformY.append((transf,data))
print(tf_transformY)

[('Logaritmo',           age  sex       bmi        bp        s1        s2        s3  \
0   -0.405465  0.0 -0.540178 -0.599118 -1.223775 -1.358788 -1.571217   
1   -0.727049 -inf -1.905419 -1.043804 -0.863773 -1.181648 -0.472604   
2   -0.124053  0.0 -0.660624 -0.828693 -1.240583 -1.351066 -1.399366   
3   -2.484907 -inf -1.198478 -1.171637 -0.702999 -0.804724 -1.453434   
4   -0.660357 -inf -1.576915 -0.599118 -0.764243 -0.873876 -0.942608   
..        ...  ...       ...       ...       ...       ...       ...   
437 -0.380772  0.0 -0.863965 -0.350657 -0.840783 -1.022869 -1.348073   
438 -0.762140  0.0 -1.254831 -1.697731 -0.466090 -0.478807 -1.348073   
439 -0.380772  0.0 -1.254831 -0.633904 -1.143733 -1.127922 -1.299283   
440 -1.261131 -inf -0.701446 -0.766172 -0.673729 -0.876266 -1.348073   
441 -1.261131 -inf -2.716349 -2.065455 -0.287682 -0.784878 -0.026317   

           s4        s5        s6  
0   -1.265538 -0.575867 -0.822359  
1   -1.958685 -1.503084 -1.791759  
2   -1.26553

C:\Users\Jhonier\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jhonier\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


#### Realizamos la regresión lineal con base en las variables obtenidas por el mejor modelo.
###### Y utilizamos el error cuadrático medio (ECM) como estimador de precisión

In [41]:
#Regresion Lineal
'''regressor = LinearRegression()
regressor.fit(X_train[["sex","bmi","bp","s1","s4","s5"]].copy(), y_train) # Entrenamiento
Y_pred = regressor.predict(X_test[["sex","bmi","bp","s1","s4","s5"]].copy()) # Pronostico
ecm = mean_squared_error(y_test ,Y_pred) #ECM
print("ECM de pronostico", ecm)
plt.figure(figsize=(8, 8))
plt.title("Grafica de distribución de los datos para el modelo de regresión lineal")
plt.plot(y_test, Y_pred, 'bo',color='green')
plt.plot([0, max(y_test)], [0, max(y_test)], color='black') '''

transformaciones = [
  ("Exponencial", lambda x: np.exp(x)),
  ("Identidad", lambda x: x),
  ("Raiz cuadrada", lambda x: x**(1/2)),
  ("Raiz cúbica", lambda x: x**(1/3))
]


for nombre, func in transformaciones:
    
    datax = datos_normX.apply(func)
    datay = datos_normY.apply(func)
    #data = data.apply(func)
    #df = diabetes_df.apply(trans)
    X_train, X_test, y_train, y_test = train_test_split(datax, 
                                                        datay,
                                                        test_size=0.20)
    
    lm = LinearRegression()
    lm.fit(X_train,y_train)
    predictions = lm.predict(X_test)
    print( nombre )
    print('MAE:', metrics.mean_absolute_error(y_test, predictions))
    print('MSE:', metrics.mean_squared_error(y_test, predictions))
    print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
    print(" \n")




Exponencial
MAE: 0.2341914531597468
MSE: 0.08572804543238643
RMSE: 0.292793520133876
 

Identidad
MAE: 0.14683370736244142
MSE: 0.031860886728951016
RMSE: 0.17849618127274045
 

Raiz cuadrada
MAE: 0.13077449131117766
MSE: 0.024679365451943408
RMSE: 0.1570966754961524
 

Raiz cúbica
MAE: 0.09086262951204574
MSE: 0.01442430193219429
RMSE: 0.12010121536518392
 



###### Los errores más pequeños se producen con la transformación raíz cúbica, por lo tanto, se utilizará para aplicarla a los datos

### Modelo de redes neuronales
Utilizaremos un perceptron multicapa (MLPRegressor) como modelo y encontraremos los parámetros óptimos utilizando la libreria GridSearchCV de SkLearn, el cuál usa validación cruzada para optimizar los parámetros.

In [86]:
datax = datos_normX.apply(transformaciones[3][1])
datay = datos_normY.apply(transformaciones[3][1])
#data = data.apply(func)
#df = diabetes_df.apply(trans)
X_train, X_test, y_train, y_test = train_test_split(datax, 
                                                    datay,
                                                    test_size=0.20)

#Utilizamos un perceptrón multicapa
PMC = MLPRegressor(max_iter=15000)

In [87]:
parametros = {
    'hidden_layer_sizes': [(2,3),(4,), (7,), (10,),(3,2)],
    'activation': ['logistic', 'relu'],
    'solver': ['lbfgs','sgd'], 
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant'],
}

In [88]:
clf = GridSearchCV(PMC, parametros,cv=3)
clf.fit(X_train, y_train)

C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-v

C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-v

C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-v

C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jhonier\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-v

GridSearchCV(cv=3, error_score=nan,
             estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                    batch_size='auto', beta_1=0.9, beta_2=0.999,
                                    early_stopping=False, epsilon=1e-08,
                                    hidden_layer_sizes=(100,),
                                    learning_rate='constant',
                                    learning_rate_init=0.001, max_fun=15000,
                                    max_iter=15000, momentum=0.9,
                                    n_iter_no_change=10,
                                    nesterovs_momentum=True, power_t=0.5,
                                    random_stat...
                                    solver='adam', tol=0.0001,
                                    validation_fraction=0.1, verbose=False,
                                    warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'activation': ['logistic', '

In [89]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'activation': 'logistic', 'alpha': 0.0001, 'hidden_layer_sizes': (2, 3), 'learning_rate': 'constant', 'solver': 'lbfgs'}


###### Los mejores parámetros encontrados son:
* Función de activación: logistic
* alpha : 0.0001
* neuronas:2
* capas 3
* solver: lbfgs

In [95]:

print('Score:', round(clf.score(X_test, y_test),2))


Score: 0.4
